In [5]:
from google.cloud import bigquery

project_id="students-group2"
client = bigquery.Client(project=project_id)

query = """
SELECT userId, movieId, rating, timestamp
FROM `master-ai-cloud.MoviePlatform.ratings`
"""
df_ratings = client.query(query).to_dataframe()


In [6]:
df_ratings

,userId,movieId,rating,timestamp
0,1,204,0.5,1217895786
1,1,256,0.5,1217895764
2,1,277,0.5,1217895772
3,1,719,0.5,1217895799
4,1,45950,0.5,1217897813
...,...,...,...,...
105334,668,93040,5.0,1331051757
105335,668,98154,5.0,1353208964
105336,668,101862,5.0,1373168467
105337,668,106916,5.0,1388740601


In [7]:
# Lister toutes les tables dans le dataset master
dataset_id = "master-ai-cloud.MoviePlatform"
tables = list(client.list_tables(dataset_id))

for table in tables:
    print(table.table_id)

movies
ratings


In [8]:
from google.cloud import bigquery
project_id = "students-group2"
dataset_name = "fatma_aziz"

dataset_id = f"{project_id}.{dataset_name}"

dataset = bigquery.Dataset(dataset_id)

dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)
print(f"Dataset créé : {dataset.dataset_id}")

Dataset créé : fatma_aziz


In [9]:
df_movies = client.query("""
SELECT movieId, title, genres
FROM `master-ai-cloud.MoviePlatform.movies`
""").to_dataframe()

In [10]:
df_movies

,movieId,title,genres
0,126929,Li'l Quinquin ( ),(no genres listed)
1,135460,Pablo (2012),(no genres listed)
2,138863,The Big Broadcast of 1936 (1935),(no genres listed)
3,141305,Round Trip to Heaven (1992),(no genres listed)
4,141472,The 50 Year Argument (2014),(no genres listed)
...,...,...,...
10324,85896,Tribute to a Bad Man (1956),Western
10325,103570,Dead Man's Burden (2012),Western
10326,105223,Colorado Territory (1949),Western
10327,128360,The Hateful Eight (2015),Western


In [15]:
from google.cloud import bigquery

client = bigquery.Client()

# Vider ratings
client.query("DELETE FROM `students-group2.fatma_aziz.ratings` WHERE TRUE").result()

# Vider movies
client.query("DELETE FROM `students-group2.fatma_aziz.movies` WHERE TRUE").result()

print("Tables vidées avec succès !")


Tables vidées avec succès !


In [16]:
# TABLE 1 : ratings
table_id_ratings = "students-group2.fatma_aziz.ratings"

job = client.load_table_from_dataframe(df_ratings, table_id_ratings)
job.result()

print("Table ratings copiée avec succès !")


# TABLE 2 : movies  (⚠️ nouveau nom)
table_id_movies = "students-group2.fatma_aziz.movies"

job = client.load_table_from_dataframe(df_movies, table_id_movies)
job.result()

print("Table movies copiée avec succès !")


Table ratings copiée avec succès !
Table movies copiée avec succès !


In [17]:
client.list_tables("students-group2.fatma_aziz")

In [18]:
df = client.query("SELECT COUNT(*) FROM `students-group2.fatma_aziz.ratings`").to_dataframe()
df


,f0_
0,105339


In [19]:
df_1 = client.query("SELECT COUNT(*) FROM `students-group2.fatma_aziz.movies`").to_dataframe()
df_1

,f0_
0,10329
